## steps i will follow
- learn to load a model locally give a prompt to and check how it gives output
- design a pipeline  
- design a prompt 
- storing the answer 

let's us check the our model compatible or not 

In [5]:
import torch 
gpu_mem=torch.cuda.get_device_properties(0).total_memory
gpu_mem=gpu_mem/2**30
gpu_mem

3.99951171875

In [6]:
!nvidia-smi

Sat Jun 14 04:44:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.57                 Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   32C    P0             10W /   70W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### get access of gemma model 1b it 
- https://huggingface.co/google/gemma-3-1b-it

### download the hugging face cli to directly login from terminal 
- https://huggingface.co/docs/huggingface_hub/en/guides/cli

## requirement for flash attention 2 

- Requiremennt------- Minimum Version ---------------------mine version
- CUDA	      ------- 11.6 or later- ----------------------- 12.9
- PyTorch     ------- 2.0 or later --------------------------- 2.7.0+cu118
- NVIDIA GPU ------- Ampere or newer (A100, A10, H100, etc.)------rtx 2050
- Python	------- 3.8+       ---------------------------  3.12.8

## installing flash attention for attention architecture

- https://github.com/Dao-AILab/flash-attention
-  Make sure that ninja is installed and that it works correctly ---->given in above github link req for installation of flask 
- will take around 8 hour to compile 
- version   flash_attn-2.5.5


In [7]:
import torch 
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available


c:\Users\ALOK\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))#------>(8,6) this means it is compute capability is 8.6
print(torch.cuda.get_device_properties(0))

NVIDIA GeForce RTX 2050
(8, 6)
_CudaDeviceProperties(name='NVIDIA GeForce RTX 2050', major=8, minor=6, total_memory=4095MB, multi_processor_count=16, uuid=e4e4042f-c536-1e4f-3663-67440a04ba8f, L2_cache_size=1MB)


## error i facead while loading this model
- use this specific transformer model ------------> pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3 <br>
  even latest transformer model did not suppport soo use this only
- i was accidentally trying to load 2b model and my kernel crashed

- if u are using quantization config then do not set low_cpu_mem_usage=False ----> in llm_model


the reason i am not using quantization config cause <br>
model is ending up on the CPU, but i am trying to use fp4 quantization, which is only supported on GPU by the bitsandbytes library.<br>
but since this bits and and bytes i am using is not cuda supported cause my cuda version is 12.9.<br>
>version is newer than what bitsandbytes officially supports. The latest prebuilt bitsandbytes wheel currently supports up to CUDA 12.0 — 
>there is no official bitsandbytes-cuda129 wheel yet.

> i have have a CPU-only version of bitsandbytes installed — which does not support GPU quantization types like fp4 or int4.

### i tried above by making use_quantization false but geting this error
-Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method<br>        wrapper_CUDA__index_select)
 
- i am running outof gpu memory that i suppose cause i tried all possible way to but failed .(btw my both input id and llm model is on cuda stil this error)

In [9]:
quantization_config= BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype=torch.float16)
use_quantization_config=True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attention_implementation= 'flash_attention_2'
model_id='google/gemma-3-1b-it'
tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                        torch_dtype=torch.float16,
                                        quantization_config=quantization_config if use_quantization_config else None,
                                        attn_implementation=attention_implementation)
if not quantization_config:
    llm_model.to('device')

`low_cpu_mem_usage` was None, now default to True since model is quantized.


## below to check flash attention 2 is used or not 

In [10]:

print(llm_model.config._attn_implementation)


flash_attention_2


In [100]:
llm_model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear4bit(in_features=1152, out_features=1024, bias=False)
          (k_proj): Linear4bit(in_features=1152, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=1152, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=1024, out_features=1152, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear4bit(in_features=1152, out_features=6912, bias=False)
          (up_proj): Linear4bit(in_features=1152, out_features=6912, bias=False)
          (down_proj): Linear4bit(in_features=6912, out_features=1152, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_l

In [ ]:
import pandas as pd
df=pd.read_csv(r"D:\summer project\Fire\final_dataset\twittter_final.csv")

In [157]:
data=[]
for pos,row in df.iterrows():
    data.append(row.to_dict())

In [160]:
def promt_generator(num:int,data:list[dict]):
    tweet=data[num]["cleaned_tweet"]
    language=data[num]["language"]
    label_1=data[num]["L1_Datatype"]
    label_2=data[num]["L2_Datatype"]
    label_3=data[num]["L3_Datatype"]
    label_final=data[num]["Datatype"]

    base_prompt='''
you have specialized at detecting sentence emotion.we are playing a game in which i will give you a sentence and it's suggested emotion which we call datatype . datatype 
will be analyzed at 3 level and you have to anlyze it emotion as per suggested emotion at each level and has to respond strictly in binary whether you Agree or Disagree
CRITICAL INSTRUCTIONS:
- OUTPUT ONLY: "Agree" or "Disagree"
- NO EXPLANATIONS: Do not provide reasoning or additional text
- you have to strictly give some output and cannot leave output blank 
- strictly analyze each level and if you Disagree at any level give response in binary as disagree and if you agree at each level respond agree\n

CLASSIFICATION HIERARCHY:
Level 1 (Primary Classification) - MANDATORY
- NOISE: Unintelligible text, random characters, excessive symbols, spam, gibberish
- OBJECTIVE: Factual statements, news reports, informational content without personal opinion
- SUBJECTIVE: Personal opinions, emotions, feelings, subjective experiences\n

Level 2 (Sentiment Classification) - ONLY if Level 1=SUBJECTIVE
-  NEUTRAL: No clear positive/negative sentiment, balanced opinion, uncertainty
-  NEGATIVE: Criticism, complaints, dissatisfaction, negative emotions
- POSITIVE: Praise, satisfaction, positive emotions, recommendations

Level 3 (Neutral Subcategories) - ONLY if Level 2=NEUTRAL
- NEUTRAL SENTIMENTS: Balanced opinions, mild preferences, uncertain feelings
- QUESTIONS: Inquiries, requests for information, interrogative sentences
- ADVERTISEMENTS: Promotional content, sales pitches, marketing messages
- MISCELLANEOUS: Greetings, well-wishes, general social interactions

final level -
this level indicate wahat is overall emotion of a text sentence\n



VALIDATION RULES:\n
if any level label is blank that means you don't have to analyze those level.\n
if language= en that means it's English language

INPUT FORMAT:
Sentence: [TEXT_TO_ANALYZE]
Language: [LANGUAGE_CODE]
Level 1 Label: NOISE OR OBJECTIVE OR SUBJECTIVE
Level 2 Label: NUETRAL OR NEGATIVE OR POSITIVE OR BLANK
Level 3 Label: NUETRAL SENTIMENT OR QUESTION OR ADVERTISEMENT OR MISCELLANEOUS OR BLANK 
final level: NOISE OR OBJECTIVE OR SUBJECTIVE OR NUETRAL OR NEGATIVE OR POSITIVE OR BLANK OR NUETRAL SENTIMENT OR QUESTION OR ADVERTISEMENT OR MISCELLANEOUS OR BLANK

DETAILED EXAMPLES:
CORRECT CLASSIFICATIONS:
Example 1: NOISE
- Sentence: "asdfgh !@#$%^& "
- Language: en
- Level 1: noise, Level 2: [blank], Level 3: [blank]
- final level-noise
- Expected Output: AGREE\n

Example 2: positive in SUBJECTIVE
- Sentence: "This restaurant series the most amazing pasta I've ever tasted!"
- Language: en
- Level 1: subjective, Level 2: positive, Level 3: [blank]
final level- positive
- Expected Output: AGREE\n

Example 3: Objective mislabeled as Subjective
- Sentence: "Apple Inc. reported quarterly earnings of $89.5 billion."
- Language: en
- Level 1: subjective, Level 2:positive, Level 3: [blank]
final level - positive
- Expected Output: DISAGREE\n

Example 4: Negative sentiment mislabeled as Positive
- Sentence: "This movie was boring and a complete waste of time."
- Language: en
- Level 1: 2, Level 2: 2, Level 3: [blank]
final level - positive
- Expected Output: DISAGREE

EVALUATION PROCESS:
1. Read and understand the sentence in the specified language
2. Analyze content type: Is it noise, factual, or opinion-based?
3. Assess sentiment: If subjective, determine positive/negative/neutral tone
4. Check subcategories: If neutral, identify specific type
5. Validate hierarchy: Ensure labels follow the hierarchical rules
6. Binary decision: AGREE if all labels are correct, DISAGREE if any label is wrong\n


Now the game starts from here analyze each level of below input answer strictly inAgree or Disagree and strictly follow rule\n
: here is your sentence and each level datatype\n
Sentence: {tweet}\n
Language: {language}\n
Level 1 Label: {label_1}\n
Level 2 Label: {label_2}\n
Level 3 Label: {label_3}\n
final label: {label_final}\n

'''
    prompt=base_prompt.format(tweet=tweet,language=language,label_1=label_1,label_2=label_2,label_3=label_3,label_final=label_final)
    return prompt


In [151]:
# # input_text="what are macronutrient? and what role they play in human body?"
# # print(f"input text:{query}")
# dailogue_template=[
#     {
#         "role":"user",
#         "content": query
#     }
# ]
# prompt=tokenizer.apply_chat_template(conversation=dailogue_template,tokenize=False,add_generation_prompt=True)
# print(f"format template:\n{query}")

In [150]:
# input_id=tokenizer(prompt,return_tensors='pt').to(device)
# input_id

In [149]:

# output=llm_model.generate(**input_id,max_new_tokens=60)
# decode_output=tokenizer.decode(output[0])
# pattern = r"<bos><bos><start_of_turn>user.*?<start_of_turn>model"
# cleaned_output = re.sub(pattern, "", decode_output, flags=re.DOTALL)
# print(f"output: {cleaned_output}")

In [1]:
import re
def Pipeline(total_data_num:int,max_gen_token:int):
    query=promt_generator(num=total_data_num,data=data)
    dailogue_template=[
        {
            "role":"user",
            "content": query
        }
    ]
    prompt=tokenizer.apply_chat_template(conversation=dailogue_template,tokenize=False,add_generation_prompt=True)
    input_id=tokenizer(prompt,return_tensors='pt').to(device)
    output=llm_model.generate(**input_id,max_new_tokens=max_gen_token)
    decode_output=tokenizer.decode(output[0])
    pattern = r"<bos><bos><start_of_turn>user.*?<start_of_turn>model"
    cleaned_output = re.sub(pattern, "", decode_output, flags=re.DOTALL)
    print(cleaned_output)




In [ ]:
for i in range(3):
    Pipeline(total_data_num=i,max_gen_token=512)


Okay, analyzing the sentence "i vote for at this year is"

Level 1 Label: Noise
Level 2 Label: nan
Level 3 Label: nan<end_of_turn>

Okay, let's analyze the provided sentence.

Sentence: start with what is right rather than what is acceptable. -franz kafka
Language: en
Level 1 Label: Noise
Level 2 Label: nan
Level 3 Label: nan

Final Label: Noise
<end_of_turn>

Okay, here's my analysis of the provided sentence:

Sentence: "ain the same day i saw & i had an autograph i votea"
Language: en
Level 1 Label: Noise
Level 2 Label: NaN
Level 3 Label: NaN
final label:
